In [188]:
import pandas as pd
import json
from collections import Counter, defaultdict
from datetime import datetime

In [160]:
# irrelevant questions
USELESS_QS = "6 7 27 31 53 81 113 114".split()

qs = pd.read_csv("Questions.txt", sep="|", dtype=str)

ans = pd.read_csv("Answers.txt", sep="|", dtype=str)
ans["Descr"] = ans["Descr"].apply(lambda _: _.split(";")[-1].strip())

resp = pd.read_csv("profileresponses.txt", sep="|", encoding='latin-1', dtype=str)
resp = resp[~resp.Question_PK.isin(USELESS_QS)]
resp['ResponseText'] = resp['ResponseText'].fillna('')
resp['ResponseDate'] = resp['ResponseDate'].fillna('')

In [565]:
craigs_resp = resp[resp.Member_PK == '5200247']
craigs_resp

,Member_PK,Question_PK,Answer_PK,ResponseText,ResponseDate,RespondedTime
375769,5200247,1,2,,,Sep 25 2017 1:20PM
724046,5200247,2,3,,Mar 15 1994 12:00AM,Sep 25 2017 1:20PM
1087498,5200247,3,5,,,Sep 25 2017 1:20PM
1449434,5200247,4,8,4570,,Sep 25 2017 1:20PM
3751480,5200247,11,31,,,Sep 25 2017 1:20PM
3887583,5200247,12,33,female,Nov 1 2016 12:00AM,Sep 25 2017 1:21PM
5013577,5200247,25,57,,,Sep 25 2017 1:21PM
5944963,5200247,28,80,,,Sep 25 2017 1:21PM
6206172,5200247,29,110,,,Sep 25 2017 1:22PM
6468556,5200247,30,122,,,Sep 25 2017 1:22PM


In [648]:
class RespCollector(object):
    
    def __init__(self, df):
        self.info_dict = defaultdict(lambda: defaultdict())
        self.df = df
        self.kids_dict = defaultdict(lambda : defaultdict)
        self.work_dict = defaultdict()
        self.finance_dict = defaultdict()
        self.insurance_dict = defaultdict()

    def _format_date(self, date_str):
        return datetime.strptime(" ".join(date_str.split()[:-1]), "%b %d %Y").strftime("%d/%m/%Y")
      
    def _add_top_level_answer(self, dk, ki, answer, mx=1):
        assert len(answer) == 1, 'ERROR: _add_top_level_answer only accepts a single-element list!'
        if ki not in self.info_dict:
            dk[ki] = [answer[0]]
        else:
            dk[ki].append(answer[0])
        if mx == 1:
            dk[ki] = dk[ki][0]
            
        return self
    
    def _get_answer(self, row):
        """
        row has to be a row object
        """
        question = qs.loc[qs.Question_PK == row[1].Question_PK, "Descr"].values[0].lower().strip()
    
        answer = []
    
        if row[1].ResponseText:
            answer.append(row[1].ResponseText.lower().strip())
        if row[1].ResponseDate:
            answer.append(self._format_date(row[1].ResponseDate))
    
        if not answer:
            answer.append(ans.loc[(ans.Answer_PK == row[1].Answer_PK) & (ans.Question_PK == row[1].Question_PK), "Descr"].values[0].strip().lower())
    
        if int(row[1].Question_PK) == 1:
            self._add_top_level_answer(self.info_dict, "gender", answer)
        
        if int(row[1].Question_PK) == 2:
            self._add_top_level_answer(self.info_dict, "dob", answer)
    
        if int(row[1].Question_PK) == 3:
            self.info_dict["marital_status"] = answer
    
        if int(row[1].Question_PK) == 4:
            self._add_top_level_answer(self.info_dict, "home_postcode", answer)
    
        if int(row[1].Question_PK) == 6:
            self._add_top_level_answer(self.info_dict, "lives_in_state", answer)
    
        if int(row[1].Question_PK) == 7:
            self._add_top_level_answer(self.info_dict, "lives_in_area", answer)
    
        if int(row[1].Question_PK) == 8:
            self._add_top_level_answer(self.info_dict, "people_in_household", answer)
    
        if int(row[1].Question_PK) == 9:
            self._add_top_level_answer(self.info_dict, "housing_type", answer)
    
        if int(row[1].Question_PK) == 10:
            self._add_top_level_answer(self.info_dict, "household_type", answer)
        
        # ----- K I D S
        
        if int(row[1].Question_PK) == 11:
            self._add_top_level_answer(self.kids_dict, "kids_u18_in_household", answer)
        
        if int(row[1].Question_PK) == 77:
            self._add_top_level_answer(self.kids_dict, "total_kids", answer)
        
        if int(row[1].Question_PK) == 113:
            self._add_top_level_answer(self.kids_dict, "is_pregnant", answer)
        
        if int(row[1].Question_PK) == 114:
            self._add_top_level_answer(self.kids_dict, "baby_due", answer)
        
        if int(row[1].Question_PK) in range(12,22):
            if "kids_info" not in self.kids_dict:
                self.kids_dict["kids_info"] = [{"gender": answer[0], "dob": answer[1]}]
            else:
                self.kids_dict["kids_info"].append({"gender": answer[0], "dob": answer[1]})               
            
        if int(row[1].Question_PK) == 23:
            self._add_top_level_answer(self.info_dict, "wants_be_in_focus_group", answer)
        
        if int(row[1].Question_PK) == 24:
            self._add_top_level_answer(self.info_dict, "wants_be_in_phone_interview", answer)
        
        if int(row[1].Question_PK) == 25:
            self._add_top_level_answer(self.info_dict, "education", answer)
        
        if int(row[1].Question_PK) == 26:
            self._add_top_level_answer(self.info_dict, "main_salary_earner", answer)
        
        # ----- W O R K
        
        if int(row[1].Question_PK) == 5:
            self.work_dict["work_postcode"] = answer
        
        if int(row[1].Question_PK) == 27:
            self.work_dict["employment_status"] = [answer, self._format_date(row[1].RespondedTime)]
        
        if int(row[1].Question_PK) == 28:
            self.work_dict["industry"] = [answer, self._format_date(row[1].RespondedTime)]
        
        if int(row[1].Question_PK) == 29:
            self.work_dict["occupation"] = [answer, self._format_date(row[1].RespondedTime)]
        
        if int(row[1].Question_PK) == 30:
            self.work_dict["company_size"] = [answer, self._format_date(row[1].RespondedTime)]
        
        if int(row[1].Question_PK) == 31:
            self.work_dict["company_annual_turnover"] = [answer, self._format_date(row[1].RespondedTime)]
        
        # ----- F I N A N C E
        
        if int(row[1].Question_PK) == 32:
            self._add_top_level_answer(self.finance_dict, "annual_income", answer)
        
        if int(row[1].Question_PK) == 33:
            self._add_top_level_answer(self.finance_dict, "annual_household_income", answer)
        
        if int(row[1].Question_PK) == 34:
            self._add_top_level_answer(self.finance_dict, "ways_to_pay_bills", answer, 5)
        
        if int(row[1].Question_PK) == 35:
            self._add_top_level_answer(self.finance_dict, "financial_services", answer, 5)
        
        if int(row[1].Question_PK) == 36:
            self.info_dict["financial_institutions"] = answer
        
        if int(row[1].Question_PK) == 37:
            self._add_top_level_answer(self.finance_dict, "main_financial_institutions", answer, 5)
        
        if int(row[1].Question_PK) == 38:
            self._add_top_level_answer(self.finance_dict, "numb_credit_store_cards", answer)
        
        if int(row[1].Question_PK) == 39:
            self._add_top_level_answer(self.finance_dict, "total_credit_limit", answer)
        
        # ----- I N S U R A N C E
        
        if int(row[1].Question_PK) == 40:
            self._add_top_level_answer(self.insurance_dict, "insurance_policies", answer, 5)
        
        if int(row[1].Question_PK) == 66:
            self._add_top_level_answer(self.insurance_dict, "has_health_insurance", answer)
            
        if int(row[1].Question_PK) == 67:
            self._add_top_level_answer(self.insurance_dict, "private_health_insurance_with", answer, 5)
        
        if int(row[1].Question_PK) == 41:
            self._add_top_level_answer(self.info_dict, "home_ownership_status", answer)
        
        if int(row[1].Question_PK) == 42:
            self.info_dict["vehicle_owned"] = answer
        
        if int(row[1].Question_PK) == 43:
            self._add_top_level_answer(self.info_dict, "cond_most_used_vehicle_when_purchased", answer)
        
        if int(row[1].Question_PK) == 44:
            self.info_dict["vehicle_makes_owned"] = answer
        
        if int(row[1].Question_PK) == 45:
            self.info_dict["value_most_used_vehicle_when_purchased"] = answer
        
        if int(row[1].Question_PK) == 46:
            self.info_dict["vehicle_types_owned"] = answer
        
        if int(row[1].Question_PK) == 47:
            self.info_dict["main_transport_to_work]"] = answer
        
        if int(row[1].Question_PK) == 48:
            self.info_dict["owns_mobile"] = answer
        
        if int(row[1].Question_PK) == 49:
            self.info_dict["mobile_brand"] = answer
        
        if int(row[1].Question_PK) == 50:
            self.info_dict["who_pays_mobile"] = answer
        
        if int(row[1].Question_PK) == 51:
            self.info_dict["mobile_network"] = answer
        
        if int(row[1].Question_PK) == 52:
            self.info_dict["mobile_on_contract"] = answer
        
        if int(row[1].Question_PK) == 53:
            self.info_dict["mobile_contract_expiration"] = answer
        
        if int(row[1].Question_PK) == 54:
            self.info_dict["internet_at_home"] = answer
        
        if int(row[1].Question_PK) == 55:
            self.info_dict["type_internet_at_home"] = answer
        
        if int(row[1].Question_PK) == 56:
            self.info_dict["isp"] = answer
        
        if int(row[1].Question_PK) == 57:
            self.info_dict["owns_computer"] = answer
                
        if int(row[1].Question_PK) == 58:
            self.info_dict["computer_type"] = answer
            
        if int(row[1].Question_PK) == 59:
            self.info_dict["landline_at_home"] = answer
        
        if int(row[1].Question_PK) == 60:
            self.info_dict["landline_provider"] = answer
            
        if int(row[1].Question_PK) == 61:
            self.info_dict["online_purchasing_freq"] = answer
            
        if int(row[1].Question_PK) == 62:
            self.info_dict["owns_devices"] = answer
            
        if int(row[1].Question_PK) == 63:
            self._add_top_level_answer(self.info_dict, "smoker", answer)
        
        if int(row[1].Question_PK) == 64:
            self.info_dict["wears_glasses_or_lenses"] = answer
        
        if int(row[1].Question_PK) == 65:
            self.info_dict["conditions_suffered"] = answer
        

        

        
        if int(row[1].Question_PK) == 68:
            self.info_dict["role_in_buying_groceries"] = answer
            
        if int(row[1].Question_PK) == 69:
            self.info_dict["buying_groceries_online"] = answer
        
        if int(row[1].Question_PK) == 70:
            self._add_top_level_answer(self.info_dict, "born_in", answer)
        
        if int(row[1].Question_PK) == 71:
            self.info_dict["ancestry"] = answer
        
        if int(row[1].Question_PK) == 72:
            self.info_dict["languages_at_home"] = answer
        
        if int(row[1].Question_PK) == 73:
            self.info_dict["religion"] = answer
        
        if int(row[1].Question_PK) == 74:
            self.info_dict["pets"] = answer
        
#         if int(row[1].Question_PK) == 75:
#             self.info_dict["pay_tv_at_home"] = answer
        
#         if int(row[1].Question_PK) == 76:
#             self.info_dict["pay_tv_provider"] = answer
        
#         if int(row[1].Question_PK) == 78:
#             self._add_top_level_answer(self.info_dict, "okayed_kids_online_surveys", answer)
        
#         if int(row[1].Question_PK) == 79:
#             self._add_top_level_answer(self.info_dict, "wants_sms_offers", answer)
        
#         if int(row[1].Question_PK) == 80:
#             self._add_top_level_answer(self.info_dict, "mobile_number", answer)
        
#         if int(row[1].Question_PK) == 81:
#             self.info_dict["vehicle_insurance_expiration"] = answer
        
#         if int(row[1].Question_PK) == 82:
#             self.info_dict["owns_swimming_pool"] = answer
        
#         if int(row[1].Question_PK) == 83:
#             self.info_dict["interested_in_activities"] = answer
        
#         if int(row[1].Question_PK) == 84:
#             self.info_dict["home_building_insurance_expiration"] = answer
        
        if int(row[1].Question_PK) == 85:
            self.info_dict["home_contents_insurance_expiration"] = answer
        
        if int(row[1].Question_PK) == 86:
            self.info_dict["life_insurance_expiration"] = answer
        
        if int(row[1].Question_PK) == 87:
            self.info_dict["health_insurance_expiration"] = answer
        
        if int(row[1].Question_PK) == 88:
            self.info_dict["boat_insurance_expiration"] = answer
        
        if int(row[1].Question_PK) == 89:
            self.info_dict["caravan_insurance_expiration"] = answer
        
        if int(row[1].Question_PK) == 90:
            self.info_dict["bought_home_in"] = answer
        
        if int(row[1].Question_PK) == 91:
            self.info_dict["total_vehicles_in_hhold"] = answer
      
        if int(row[1].Question_PK) == 92:
            self.info_dict["year_bought_most_used_vehicle"] = answer
        
        if int(row[1].Question_PK) == 93:
            self.info_dict["social_networks"] = answer
        
        if int(row[1].Question_PK) == 94:
            self.info_dict["main_supermarkets_for_groceries"] = answer
        
        if int(row[1].Question_PK) == 95:
            self.info_dict["regularly_shops_at_department_stopes"] = answer
        
        if int(row[1].Question_PK) == 96:
            self.info_dict["member_of_frequent_flyer"] = answer
        
        if int(row[1].Question_PK) == 97:
            self.info_dict["flights_past_12_months"] = answer
        
        if int(row[1].Question_PK) == 98:
            self.info_dict["purpose_flying_past_12_months"] = answer
        
        if int(row[1].Question_PK) == 99:
            self.info_dict["how_often_would_fly_for_business_a_year"] = answer
        
        if int(row[1].Question_PK) == 100:
            self.info_dict["how_often_would_fly_for_leisure_a_year"] = answer
        
        if int(row[1].Question_PK) == 101:
            self.info_dict["on_holidays_goes_to"] = answer
        
        if int(row[1].Question_PK) == 102:
            self.info_dict["rented_a_car_past_12_months"] = answer
        
        if int(row[1].Question_PK) == 103:
            self.info_dict["regular_alcoholic_drinks"] = answer
        
        if int(row[1].Question_PK) == 104:
            self.info_dict["energy_drinks"] = answer
        
        if int(row[1].Question_PK) == 105:
            self.info_dict["sports_drinks"] = answer
        
        if int(row[1].Question_PK) == 106:
            self._add_top_level_answer(self.info_dict, "bottles_wine_a_month_at_hhold", answer)
        
        if int(row[1].Question_PK) == 107:
            self._add_top_level_answer(self.info_dict, "how_much_ok_to_spend_bottle_wine", answer)
        
        if int(row[1].Question_PK) == 108:
            self._add_top_level_answer(self.info_dict, "wants_wine_offers", answer)
        
        if int(row[1].Question_PK) == 109:
            self._add_top_level_answer(self.info_dict, "reads_newspapers", answer, 5)
        
        if int(row[1].Question_PK) == 110:
            self._add_top_level_answer(self.info_dict, "reads_magazines", answer, 5)
        
        if int(row[1].Question_PK) == 111:
            self._add_top_level_answer(self.info_dict, "watches_sports", answer, 5)
        
        if int(row[1].Question_PK) == 112:
            self._add_top_level_answer(self.info_dict, "reads_news_portals", answer, 5)
        
        if int(row[1].Question_PK) == 115:
            self._add_top_level_answer(self.info_dict, "credit_card_types", answer, 5)
        
        if int(row[1].Question_PK) == 116:
            self.info_dict["plans_to_purchase"] = answer
            
        if int(row[1].Question_PK) == 117:
            self.info_dict["deviced_purchased_upgraded_past_12_month"] = answer
        
        if int(row[1].Question_PK) == 118:
            self.info_dict["type_of_cigarettes"] = answer
        
        if int(row[1].Question_PK) == 119:
            self._add_top_level_answer(self.info_dict, "brands_of_cigarettes", answer, 5)
        
        if int(row[1].Question_PK) == 120:
            self._add_top_level_answer(self.info_dict, "brands_of_cigarette_papers", answer, 5)
            
        return self
    
    def collect_answers(self):
        for row in self.df.iterrows():
            self._get_answer(row)
        
        self.info_dict["work"] = self.work_dict
        self.info_dict["kids"] = self.kids_dict
        self.info_dict["financial_situation"] = self.finance_dict
        self.info_dict["insurance"] = self.insurance_dict

In [649]:
rc = RespCollector(craigs_resp)

In [650]:
rc.collect_answers()

In [651]:
rc.info_dict

defaultdict(<function __main__.RespCollector.__init__.<locals>.<lambda>>,
            {'dob': '15/03/1994',
             'education': 'university (bachelor) degree',
             'financial_situation': defaultdict(None,
                         {'annual_household_income': '$100,001 - $130,000',
                          'annual_income': '$70,001 - $80,000'}),
             'gender': 'female',
             'home_ownership_status': 'own my home',
             'home_postcode': '4570',
             'insurance': defaultdict(None,
                         {'has_health_insurance': 'yes',
                          'private_health_insurance_with': ['medibank private']}),
             'kids': defaultdict(<function __main__.RespCollector.__init__.<locals>.<lambda>>,
                         {'kids_info': [{'dob': '01/11/2016',
                            'gender': 'female'}],
                          'kids_u18_in_household': 'yes',
                          'total_kids': '1'}),
             'mari

In [647]:
qs.loc[qs.Question_PK == '40','Descr'].values[0]

'Which, if any, of the following insurance policies do you hold?'

In [452]:
qs[99:]

,Question_PK,Topic_pK,QuestionType_PK,Descr,Sortorder,IsActive,MaxAnswers,DeselectAll
99,101,9,12,"When on holidays, do you prefer to go on ... ?",101,0,10,NaN
100,102,9,10,Have you rented a car for business or leisure ...,102,1,10,NaN
101,103,10,12,Which alcoholic beverages do you regularly drink?,103,1,20,NaN
102,104,10,12,Which energy drinks do you drink?,104,1,10,NaN
103,105,10,12,Which sports drinks do you drink?,105,1,10,NaN
104,106,10,11,How many bottles of wine are consumed in your ...,106,1,10,NaN
105,107,10,11,How much would you typically spend on a bottle...,107,1,10,NaN
106,108,10,11,Are you interested in being contacted with exc...,108,1,10,NaN
107,109,11,12,Which of the following newspapers do you read?,109,1,10,NaN
108,110,11,12,Which of the following magazines do you read?,110,1,10,NaN
